In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('insert_your_data_path', one_hot=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting /Users/urim/Data/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/urim/Data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/urim/Data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/urim/Data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
B = 50
T_H = 28
F_W = 28
h1 = 128
o_size = 10
lr = 0.001

In [3]:
'''
inputs = tf.placeholder(dtype=tf.float32, shape=[None, T_H, F_W]) #RNN: [Batch,T,F]=[50,28,28]
labels = tf.placeholder(dtype=tf.float32, shape=[None, o_size])
sequence_length = tf.placeholder(dtype=tf.int32, shape=[None])


##### CNN #####
inputs_cnn = tf.reshape(inputs, [-1,T_H,F_W,1])
#inputs_cnn = inputs.reshape([-1,T_H,F_W,1])#CNN:[Batch,height,width,channels]=[50,28,28,1]
W1_conv = tf.Variable(tf.random_normal([3,3,1,h1]))
b1_conv = tf.Variable(tf.random_normal([h1]))

conv1 = tf.nn.conv2d(inputs_cnn, filter=W1_conv, strides=[1,1,1,1], padding='SAME') + b1_conv #[50,28,28,h1], b1: shape이 conv된거의 shape으로 broadcasting됨.
conv1_relu = tf.nn.relu(conv1)

pool1 = tf.nn.max_pool(conv1_relu, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') #[B, T_H/2, F_W/2, h1]=[50,14,14,128]
flatten_cnn = tf.reshape(pool1, [-1,int(T_H/2*F_W/2*h1)]) #[B,128*14*14] = [50,128*14*14]


##### RNN #####
cell_ls = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(h1) for i in range(2)])
outputs, states = tf.nn.dynamic_rnn(cell_ls, inputs, sequence_length = sequence_length, dtype=tf.float32) #outputs: B,T,F
outputs_last_rnn = outputs[:,-1,:] #[B,H] = [50,128]


def train(outputs, labels, scope):
    with tf.name_scope(scope):
        W1 = tf.Variable(tf.random_normal([outputs.shape[1].value, o_size]))
        b1 = tf.Variable(tf.random_normal([o_size]))

        fc_linear = tf.matmul(outputs, W1) + b1

        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=fc_linear, labels=labels), axis=0)
        train_step = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)
        acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf.nn.softmax(fc_linear), axis=1), tf.argmax(labels, axis=1)), dtype=tf.float32), axis=0)
        return train_step, loss, acc

cnn_tensors = train(flatten_cnn, labels, 'cnn')
rnn_tensors = train(outputs_last_rnn, labels, 'rnn')
'''

In [3]:
###same shape, lr test(cnn과 rnn lr을 다르게, 최종 output의 size를 같게 돌리기)

inputs = tf.placeholder(dtype=tf.float32, shape=[None, T_H, F_W]) #RNN: [Batch,T,F]=[50,28,28]
labels = tf.placeholder(dtype=tf.float32, shape=[None, o_size])
sequence_length = tf.placeholder(dtype=tf.int32, shape=[None])


##### CNN #####
inputs_cnn = tf.reshape(inputs, [-1,T_H,F_W,1])
#inputs_cnn = inputs.reshape([-1,T_H,F_W,1])#CNN:[Batch,height,width,channels]=[50,28,28,1]
W1_conv = tf.Variable(tf.random_normal([3,3,1,32]))
b1_conv = tf.Variable(tf.random_normal([32]))
W2_conv = tf.Variable(tf.random_normal([3,3,32,16]))
b2_conv = tf.Variable(tf.random_normal([16]))

conv1 = tf.nn.conv2d(inputs_cnn, filter=W1_conv, strides=[1,1,1,1], padding='SAME') + b1_conv #[50,28,28,32], b1: shape이 conv된거의 shape으로 broadcasting됨.
print('conv1 shape: ', conv1.shape)
conv1_relu = tf.nn.relu(conv1)
print('conv1_relu shape: ', conv1_relu.shape)

pool1 = tf.nn.max_pool(conv1_relu, ksize=[1,4,4,1], strides=[1,4,4,1], padding='SAME') #[50,7,7,32]
print('pool1 shape: ', pool1.shape)

conv2 = tf.nn.conv2d(pool1, filter=W2_conv, strides=[1,1,1,1], padding='SAME') + b2_conv #[50,7,7,16], b1: shape이 conv된거의 shape으로 broadcasting됨.
print('conv2 shape: ', conv2.shape)
conv2_relu = tf.nn.relu(conv2)
print('conv2_relu shape: ', conv2_relu.shape)

pool2 = tf.nn.max_pool(conv2_relu, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') #[50,4,4,16]
print('pool2 shape: ', pool2.shape)
flatten_cnn = tf.reshape(pool2, [-1, 16*4*4]) #[B,16*4*4] = [50,256] 
print('flatten_cnn shape: ', flatten_cnn.shape)

##### RNN #####
cell_ls = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(256) for i in range(2)])
outputs, states = tf.nn.dynamic_rnn(cell_ls, inputs, sequence_length = sequence_length, dtype=tf.float32) #outputs: B,T,F
print('\nrnn outputs shape: ', outputs.shape)
outputs_last_rnn = outputs[:,-1,:] #[B,H] = [50,128]
print('outputs_last_rnn shape: ', outputs_last_rnn.shape)

def train(outputs, labels, lr, scope):
    with tf.name_scope(scope):
        W1 = tf.Variable(tf.random_normal([outputs.shape[1].value, o_size]))
        b1 = tf.Variable(tf.random_normal([o_size]))

        fc_linear = tf.matmul(outputs, W1) + b1

        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=fc_linear, labels=labels), axis=0)
        train_step = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)
        acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf.nn.softmax(fc_linear), axis=1), tf.argmax(labels, axis=1)), dtype=tf.float32), axis=0)
        return train_step, loss, acc

cnn_tensors = train(flatten_cnn, labels, lr*10, 'cnn')
rnn_tensors = train(outputs_last_rnn, labels, lr, 'rnn')

conv1 shape:  (?, 28, 28, 32)
conv1_relu shape:  (?, 28, 28, 32)
pool1 shape:  (?, 7, 7, 32)
conv2 shape:  (?, 7, 7, 16)
conv2_relu shape:  (?, 7, 7, 16)
pool2 shape:  (?, 4, 4, 16)
flatten_cnn shape:  (?, 256)
rnn outputs shape:  (?, 28, 256)
outputs_last_rnn shape:  (?, 256)


In [4]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    seq_length = [T_H]*B
    
    for step in range(1000+1):
        tr_inputs, tr_labels = mnist.train.next_batch(B)
        tr_rs_inputs = tr_inputs.reshape([B,T_H,F_W])
        _, cnn_tr_loss, cnn_tr_acc = sess.run(cnn_tensors, feed_dict={inputs:tr_rs_inputs, labels:tr_labels})
        _, rnn_tr_loss, rnn_tr_acc = sess.run(rnn_tensors, feed_dict={inputs:tr_rs_inputs, labels:tr_labels, sequence_length:seq_length})
        
        val_inputs, val_labels = mnist.validation.next_batch(B)
        val_rs_inputs = val_inputs.reshape([B,T_H,F_W])
        cnn_val_loss, cnn_val_acc = sess.run([cnn_tensors[1], cnn_tensors[2]], feed_dict={inputs:val_rs_inputs, labels:val_labels})
        rnn_val_loss, rnn_val_acc = sess.run([rnn_tensors[1], rnn_tensors[2]], feed_dict={inputs:val_rs_inputs, labels:val_labels, sequence_length:seq_length})
        
        te_inputs, te_labels = mnist.test.next_batch(B)
        te_rs_inputs = te_inputs.reshape([B,T_H,F_W])
        cnn_te_loss, cnn_te_acc = sess.run([cnn_tensors[1], cnn_tensors[2]], feed_dict={inputs:te_rs_inputs, labels:te_labels})
        rnn_te_loss, rnn_te_acc = sess.run([rnn_tensors[1], rnn_tensors[2]], feed_dict={inputs:te_rs_inputs, labels:te_labels, sequence_length:seq_length})
        
        if step % 50 == 0:
            print('step: {} \ncnn_tr_loss: {:0.4f} \trnn_tr_loss: {:0.4f} \tcnn_tr_acc: {:0.4f} \trnn_tr_acc: {:0.4f}'
                  .format(step, cnn_tr_loss, rnn_tr_loss, cnn_tr_acc, rnn_tr_acc))
            print('cnn_val_loss: {:0.4f} \trnn_val_loss: {:0.4f} \tcnn_val_acc: {:0.4f} \trnn_val_acc: {:0.4f}'
                  .format(cnn_val_loss, rnn_val_loss, cnn_val_acc, rnn_val_acc))
            print('cnn_te_loss: {:0.4f} \trnn_te_loss: {:0.4f} \tcnn_te_acc: {:0.4f} \trnn_te_acc: {:0.4f}\n'
                  .format(cnn_te_loss, rnn_te_loss, cnn_te_acc, rnn_te_acc))

step: 0 
cnn_tr_loss: 308.7114 	rnn_tr_loss: 3.0040 	cnn_tr_acc: 0.0800 	rnn_tr_acc: 0.0600
cnn_val_loss: 200.6403 	rnn_val_loss: 3.0164 	cnn_val_acc: 0.0600 	rnn_val_acc: 0.1400
cnn_te_loss: 231.9065 	rnn_te_loss: 2.9577 	cnn_te_acc: 0.1000 	rnn_te_acc: 0.1800

step: 50 
cnn_tr_loss: 19.0853 	rnn_tr_loss: 0.9304 	cnn_tr_acc: 0.5400 	rnn_tr_acc: 0.6800
cnn_val_loss: 17.4713 	rnn_val_loss: 0.9882 	cnn_val_acc: 0.5400 	rnn_val_acc: 0.6400
cnn_te_loss: 14.2840 	rnn_te_loss: 0.9180 	cnn_te_acc: 0.7200 	rnn_te_acc: 0.7400

step: 100 
cnn_tr_loss: 11.2909 	rnn_tr_loss: 0.6204 	cnn_tr_acc: 0.7000 	rnn_tr_acc: 0.7800
cnn_val_loss: 8.3474 	rnn_val_loss: 0.5419 	cnn_val_acc: 0.7000 	rnn_val_acc: 0.7600
cnn_te_loss: 11.0949 	rnn_te_loss: 0.8880 	cnn_te_acc: 0.6800 	rnn_te_acc: 0.7200

step: 150 
cnn_tr_loss: 10.3511 	rnn_tr_loss: 0.7520 	cnn_tr_acc: 0.7000 	rnn_tr_acc: 0.8200
cnn_val_loss: 8.8190 	rnn_val_loss: 0.3323 	cnn_val_acc: 0.7200 	rnn_val_acc: 0.9200
cnn_te_loss: 5.9226 	rnn_te_loss: 0.4